<h1>Part 1 : Data Pre-processing</h1>
<div>In this section we build a function that accepts raw text extracted from a doc or a query and after applying normalization, tokenization, stopword removal and stemming returns an array of tokens (token stream).</div>

In [30]:
from __future__ import unicode_literals
from hazm import *
from PersianStemmer import PersianStemmer
from langdetect import detect
import nltk
# nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

def prepare_text(raw_text, lang = 'fa'):
    if(lang == 'en'):
        tokens = word_tokenize(raw_text)
        tokens = [word for word in tokens if word.isalpha()]
        tokens = [word.lower() for word in tokens]
        porter = PorterStemmer()
        prepared_text = []
        for word in tokens:
            if(tokens.count(word) < 1/15 or len(tokens) < 120):
                prepared_text.append(porter.stem(word))
        return prepared_text
    
    elif(lang == 'fa'):
        normalizer = Normalizer()
        normalized_text = normalizer.normalize(raw_text)
        tokenizer = WordTokenizer()
        tokenized_text = tokenizer.tokenize(normalized_text)
        ps = PersianStemmer()
        prepared_text = []
        for word in tokenized_text:
            if(word[0] >= "آ" and word[0] <= "ی" and (tokenized_text.count(word) < 1/15 or len(tokenized_text) < 120)):
                prepared_text.append(ps.run(word))
        return prepared_text

raw_text = input()
print(prepare_text(raw_text, 'en'))

KeyboardInterrupt: Interrupted by user

<h1>Part 2 : Indexing</h1>
<div>In this section we build a function for positional indexing and biword indexing. We then save this indexes and also we have functions for adding and deleting docs in a dynamic way meaning that you don't need to repeat the indexing process from the beginning.</div>

In [65]:
positional_index = {}
def add_positional(term, docid, position, t):
    if(term not in positional_index.keys()):
        positional_index[term] = {}
        positional_index[term][int(docid)] = {}
        positional_index[term][int(docid)][t] = [position]
    else:
        if(int(docid) not in positional_index[term].keys()):
            positional_index[term][int(docid)] = {}
            positional_index[term][int(docid)][t] = [position]
        else:
            if(t not in positional_index[term][int(docid)]):
                positional_index[term][int(docid)][t] = [position]
            else:
                positional_index[term][int(docid)][t].append(position)

In [66]:
import pandas as pd
from xml.dom import minidom
import xml.etree.ElementTree as ET

docids = []
def construct_positional_indexes(docs_path, data_format = 'xml'):
    if(data_format == 'xml'):
        mydoc = minidom.parse(docs_path)
        texts = mydoc.getElementsByTagName('text')
        titles = mydoc.getElementsByTagName('title')
        ids = mydoc.getElementsByTagName('id')
        tree = ET.parse(docs_path)
        root = tree.getroot()
        for child in root:
            for c in child:
                if(c.tag == '{http://www.mediawiki.org/xml/export-0.10/}id'):
                    docids.append(int(c.text))
        for i in range(len(texts)):
            A = prepare_text(titles[i].firstChild.data)
            B = prepare_text(texts[i].firstChild.data)
            for j in range(len(A)):
                add_positional(A[j], docids[i], j, 'title')
            for j in range(len(B)):
                add_positional(B[j], docids[i], j, 'text')
    
    elif(data_format == 'csv'):
        data = pd.read_csv(docs_path)
        descs = data.iloc[:, 1].values
        titles = data.iloc[:, 14].values
        for i in range(len(descs)):
            docids.append(i)
            A = prepare_text(titles[i], 'en')
            B = prepare_text(descs[i], 'en')
            for j in range(len(A)):
                add_positional(A[j], i, j, 'title')
            for j in range(len(B)):
                add_positional(B[j], i, j, 'text')

In [67]:
construct_positional_indexes('persian.xml')
construct_positional_indexes('ted_talks.csv', 'csv')

In [71]:
bigram_index = {}
def add_bigram(word):
    new_word = "$" + word + "$"
    for i in range(len(new_word) - 1):
        bi = new_word[i] + new_word[i + 1]
        if(bi not in bigram_index.keys()):
            bigram_index[bi] = [word]
        else:
            if(word not in bigram_index[bi]):
                bigram_index[bi].append(word)

In [72]:
def construct_bigram_indexes(docs_path, data_format = 'xml'):
    if(data_format == 'xml'):
        mydoc = minidom.parse(docs_path)
        texts = mydoc.getElementsByTagName('text')
        titles = mydoc.getElementsByTagName('title')
        for i in range(len(texts)):
            A = prepare_text(titles[i].firstChild.data)
            B = prepare_text(texts[i].firstChild.data)
            for j in range(len(A)):
                add_bigram(A[j])
            for j in range(len(B)):
                add_bigram(B[j])
                
    elif(data_format == 'csv'):
        data = pd.read_csv(docs_path)
        descs = data.iloc[:, 1].values
        titles = data.iloc[:, 14].values
        for i in range(len(descs)):
            A = prepare_text(titles[i], 'en')
            B = prepare_text(descs[i], 'en')
            for j in range(len(A)):
                add_bigram(A[j])
            for j in range(len(B)):
                add_bigram(B[j])

In [73]:
construct_bigram_indexes('persian.xml')
construct_bigram_indexes('ted_talks.csv', 'csv')

In [97]:
def add_document_to_indexes(docs_path, doc_num, file_format = 'xml'):
    if(file_format == 'xml'):
        mydoc = minidom.parse(docs_path)
        texts = mydoc.getElementsByTagName('text')
        titles = mydoc.getElementsByTagName('title')
        if(doc_num not in docids):
            docs = []
            tree = ET.parse(docs_path)
            root = tree.getroot()
            for child in root:
                for c in child:
                    if(c.tag == '{http://www.mediawiki.org/xml/export-0.10/}id'):
                        docs.append(int(c.text))
            i = docs.index(doc_num)
            A = prepare_text(titles[i].firstChild.data)
            B = prepare_text(texts[i].firstChild.data)
            for j in range(len(A)):
                add_positional(A[j], doc_num, j, 'title')

            for j in range(len(B)):
                add_positional(B[j], doc_num, j, 'text')
            docids.append(doc_num)
            
    elif(file_format == 'csv'):
        data = pd.read_csv(docs_path)
        descs = data.iloc[:, 1].values
        titles = data.iloc[:, 14].values
        if(doc_num not in docids):
            docs = []
            for i in range(len(descs)):
                docs.append(i)
            i = docs.index(doc_num)
            A = prepare_text(titles[i])
            B = prepare_text(descs[i])
            for j in range(len(A)):
                add_positional(A[j], doc_num, j, 'title')

            for j in range(len(B)):
                add_positional(B[j], doc_num, j, 'text')
            docids.append(doc_num)

In [ ]:
add_document_to_indexes('persian.xml', 3022)

In [96]:
def delete_document_from_indexes(docs_path, doc_num, file_format = 'xml'):
    if(file_format == 'xml'):
        mydoc = minidom.parse(docs_path)
        texts = mydoc.getElementsByTagName('text')
        titles = mydoc.getElementsByTagName('title')
        if(doc_num in docids):
            docs = []
            tree = ET.parse(docs_path)
            root = tree.getroot()
            for child in root:
                for c in child:
                    if(c.tag == '{http://www.mediawiki.org/xml/export-0.10/}id'):
                        docs.append(int(c.text))
            i = docs.index(doc_num)
            A = prepare_text(titles[i].firstChild.data)
            B = prepare_text(texts[i].firstChild.data)
            for j in range(len(A)):
                if(A[j] in positional_index.keys()):
                    if(doc_num in positional_index[A[j]]):
                        del positional_index[A[j]][doc_num]
                    if(len(positional_index[A[j]].keys()) == 0):
                        del positional_index[A[j]]

            for j in range(len(B)):
                if(B[j] in positional_index.keys()):
                    if(doc_num in positional_index[B[j]]):
                        del positional_index[B[j]][doc_num]
                    if(len(positional_index[B[j]].keys()) == 0):
                        del positional_index[B[j]]
            docids.remove(doc_num)
            
    elif(file_format == 'csv'):
        data = pd.read_csv(docs_path)
        descs = data.iloc[:, 1].values
        titles = data.iloc[:, 14].values
        if(doc_num in docids):
            docs = []
            for i in range(len(descs)):
                docs.append(i)
            i = docs.index(doc_num)
            A = prepare_text(titles[i])
            B = prepare_text(descs[i])
            for j in range(len(A)):
                if(A[j] in positional_index.keys()):
                    if(doc_num in positional_index[A[j]]):
                        del positional_index[A[j]][doc_num]
                    if(len(positional_index[A[j]].keys()) == 0):
                        del positional_index[A[j]]

            for j in range(len(B)):
                if(B[j] in positional_index.keys()):
                    if(doc_num in positional_index[B[j]]):
                        del positional_index[B[j]][doc_num]
                    if(len(positional_index[B[j]].keys()) == 0):
                        del positional_index[B[j]]
            docids.remove(doc_num)

In [ ]:
delete_document_from_indexes('persian.xml', 3022)

In [92]:
import json

def save_positional_index(destination):
    j = json.dumps(positional_index)
    f = open(destination,"w")
    f.write(j)
    f.close()
    
def save_bigram_index(destination):
    j = json.dumps(bigram_index)
    f = open(destination,"w")
    f.write(j)
    f.close()

In [93]:
save_positional_index('positional.json')
save_bigram_index('bigram.json')

In [94]:
def load_positional_index(source):
    with open(source) as json_file:
        positional_index = json.load(json_file)

def load_bigram_index(source):
    with open(source) as json_file:
        bigram_index = json.load(json_file)

In [ ]:
load_positional_index('positional.json')
load_bigram_index('bigram.json')

<h1>Part 3 : Index compression</h1>
<div>In this section we are going to use variable byte coding and gamma coding to compress our positional index postiong lists and compare the index size in all 3 methods. We also need to write a decode function for both variable byte coding and gamma coding technique.</div>

In [112]:
inverted_index = {}
for term in positional_index:
    inverted_index[term] = sorted(list(positional_index[term].keys()))

In [130]:
from sys import getsizeof
from objsize import get_deep_size

print(get_deep_size(inverted_index))

3004240


In [119]:
from math import log

def log2(x):
    return log(x, 2)

def unary(N):
    return N*"1" + "0"

def binary(x, l = 1): 
    s = '{0:0%db}' % l 
    return s.format(x)

def gamma(N):
    if(N == 0):
        return '0'
    n = int(log2(N)) 
    b = N - 2**(int(log2(N))) 
    l = int(log2(N)) 
    return unary(n) + binary(b, l) 
    
def gamma_coding(index):
    result = {}
    for term in index:
        posting_list = index[term]
        temp = gamma(posting_list[0])
        for i in range(1, len(posting_list)):
            temp += gamma(posting_list[i] - posting_list[i - 1])
        result[term] = temp
    return result

In [131]:
gamma_coding_inverted_index = gamma_coding(inverted_index)
print(get_deep_size(gamma_coding_inverted_index))

2378618


In [161]:
def variable_byte(N):
    b = binary(N)
    temp = ""
    k = 0
    for i in range(1, len(b) + 1):
        if(k == 7):
            temp = '0' + temp
            k = 0
        temp = b[-i] + temp
        k += 1
    for j in range(8 - k):
        temp = '0' + temp
    temp = temp[:-8] + '1' + temp[-7:]
    return temp

def variable_byte_code(index):
    result = {}
    for term in index:
        posting_list = index[term]
        temp = variable_byte(posting_list[0])
        for i in range(1, len(posting_list)):
            temp += variable_byte(posting_list[i] - posting_list[i - 1])
        result[term] = temp
    return result

In [162]:
variable_byte_coding_inverted_index = variable_byte_code(inverted_index)
print(get_deep_size(variable_byte_coding_inverted_index))

2447567


In [ ]:
def decode_gamma(code):
    
    
def decode_gamma_coding(encoded):
    result = {}
    for term in encoded:
        

<h1>Part 4 : Query correction</h1>
<div>In this section we use the bigram indexing to correct spelling errors in the query. We first use the jaccard distance to find the most likely cases of spelling correction and from that set we use the edit distant measure to find the best answer.</div>

In [132]:
def jakard_distance(word1, word2):
    w1 = "$" + word1 + "$"
    w2 = "$" + word2 + "$"
    bi1 = []
    bi2 = []
    for i in range(len(w1) - 1):
        temp = w1[i] + w1[i + 1]
        if(temp not in bi1):
            bi1.append(temp)
            
    for i in range(len(w2) - 1):
        temp = w2[i] + w2[i + 1]
        if(temp not in bi2):
            bi2.append(temp)
    
    U = []
    M = []
    for bi in (bi1 + bi2):
        if(bi not in U):
            U.append(bi)
    
    for bi in bi1:
        if(bi in bi2):
            M.append(bi)
    jakard = len(M) / len(U)
    return jakard

def edit_distance(word1, word2):
    minimum = min(len(word1), len(word2))
    maximum = max(len(word1), len(word2))
    distance = 0
    for i in range(minimum):
        if(word1[i] != word2[i]):
            distance += 1
    distance += (maximum - minimum)
    return distance

In [139]:
def correct_query(query):
    words = query.split(" ")
    correct_query = ""
    for word in words:
        if word in positional_index.keys():
            correct_query += word
            correct_query += " "
        else:
            all1 = []
            temp = "$" + word + "$"
            for i in range(len(temp) - 1):
                bi = temp[i] + temp[i + 1]
                if(bi in bigram_index):
                    for bigram in bigram_index[bi]:
                        if(bigram not in all1):
                            all1.append(bigram)
            jakard = []
            for a in all1:
                jakard.append(jakard_distance(a, word))
            J = sorted(jakard, reverse = True)
            maximum = J[9]
            candidates = []
            for i in range(len(jakard)):
                if(jakard[i] >= maximum):
                    candidates.append(all1[i])
            distance = []
            for c in candidates:
                distance.append(edit_distance(c, word))
            minimum = min(distance)
            for i in range(len(distance)):
                if(distance[i] == minimum):
                    result = candidates[i]
                    break
            correct_query += result
            correct_query += " "
                
    return correct_query[0:-1]

In [147]:
# print(correct_query('باظیابی اتلاعاط'))
# print(correct_query('اشتان شیشتان'))
# print(correct_query('moderm infornation retreval'))
print(positional_index[])

بازی اطلاعات
استان شیروان
modern institution retreat
